# **0. Importación de librerías**

In [1]:
import pandas as pd
import joblib
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB, BernoulliNB, GaussianNB

# **1. Carga, Limpieza y Vectorización de Texto**

In [10]:
# Carga del conjunto de datos
url = "https://raw.githubusercontent.com/4GeeksAcademy/naive-bayes-project-tutorial/main/playstore_reviews.csv"
try:
    df = pd.read_csv(url)
    print(f"Filas: {df.shape[0]}, Columnas: {df.shape[1]}")
except Exception as e:
    print(f"Error al cargar el dataset: {e}")

Filas: 891, Columnas: 3


# **2. Estudio de Variables y su Contenido**

In [11]:
# Elimina la columna 'package_name'
df = df.drop(columns=['package_name'])

# Elimina espacios y convierte a minúsculas
df["review"] = df["review"].str.strip().str.lower()

# División de los datos
X = df["review"]
y = df["polarity"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Transforma el texto en una matriz de recuento de palabras (CountVectorizer)
#'stop_words = "english"' se utiliza para eliminar palabras comunes sin significado
vec_model = CountVectorizer(stop_words = "english")

# Entrena el transformador en X_train y lo aplica
X_train_vec = vec_model.fit_transform(X_train)
X_test_vec = vec_model.transform(X_test)

# Convierte a array para luego utilizarlo en NB
print(f"X_train_vec shape: {X_train_vec.shape}")


X_train_vec shape: (712, 3310)


## *2.1. Preparación de datos. Entrenamiento de modelos*

In [13]:
# Multinomial Naive Bayes (MNB)
mnb_model = MultinomialNB()
mnb_model.fit(X_train_vec, y_train)
mnb_y_pred = mnb_model.predict(X_test_vec)
mnb_accuracy = accuracy_score(y_test, mnb_y_pred)

# Bernoulli Naive Bayes (BNB)
bnb_model = BernoulliNB()
bnb_model.fit(X_train_vec, y_train)
bnb_y_pred = bnb_model.predict(X_test_vec)
bnb_accuracy = accuracy_score(y_test, bnb_y_pred)

# Gaussian Naive Bayes (GNB)
# Se convierte la matriz dispersa a densa (numpy array) para este modelo.
X_train_dense = X_train_vec.toarray()
X_test_dense = X_test_vec.toarray()

gnb_model = GaussianNB()
gnb_model.fit(X_train_dense, y_train)
gnb_y_pred = gnb_model.predict(X_test_dense)
gnb_accuracy = accuracy_score(y_test, gnb_y_pred)


# Resultados comparativos

resultados = {
    'Modelo': ['MultinomialNB', 'BernoulliNB', 'GaussianNB'],
    'Accuracy': [mnb_accuracy, bnb_accuracy, gnb_accuracy]
}

df_resultados = pd.DataFrame(resultados).sort_values(by='Accuracy', ascending=False).round(4)


print("\n Comparación de los diferentes modelos de Naive Bayes")
print(df_resultados.to_string(index=False))


 Comparación de los diferentes modelos de Naive Bayes
       Modelo  Accuracy
MultinomialNB    0.8156
   GaussianNB    0.8045
  BernoulliNB    0.7709


# **3. Construcción y Comparación del Naive Bayes**

In [14]:

# Optimización del MNB
param_grid_mnb = {
    'alpha': [0.1, 0.5, 1.0, 1.5, 2.0] 
}

mnb_opt = MultinomialNB()

grid_mnb = GridSearchCV(estimator=mnb_opt, 
                        param_grid=param_grid_mnb, 
                        scoring='accuracy', 
                        cv=5, 
                        n_jobs=-1, 
                        verbose=1)

grid_mnb.fit(X_train_vec, y_train)

best_mnb_model = grid_mnb.best_estimator_
mnb_final_accuracy = accuracy_score(y_test, best_mnb_model.predict(X_test_vec))

print(f"\nMejores Hiperparámetros MNB: {grid_mnb.best_params_}")
print(f"Accuracy Final (MNB Optimizado): {mnb_final_accuracy:.4f}")

Fitting 5 folds for each of 5 candidates, totalling 25 fits



Mejores Hiperparámetros MNB: {'alpha': 0.5}
Accuracy Final (MNB Optimizado): 0.8268


## *3.1. Guardado del modelo*

In [18]:
joblib.dump(best_mnb_model, "multinomial_naive_bayes_optimizado.pkl")
print("\n✅ Modelo MNB Optimizado guardado como 'multinomial_naive_bayes_optimizado.pkl'")


✅ Modelo MNB Optimizado guardado como 'multinomial_naive_bayes_optimizado.pkl'


# **4. Exploración de alternativas**

## *4.1. Random Forest*

In [15]:
rf_model = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42, n_jobs=-1)

# Random Forest puede manejar la matriz dispersa, pero es una buena práctica pasarla a densa ya que la dispersa genera errores.
X_train_dense = X_train_vec.toarray()
X_test_dense = X_test_vec.toarray()

rf_model.fit(X_train_dense, y_train)
rf_y_pred = rf_model.predict(X_test_dense)
rf_accuracy = accuracy_score(y_test, rf_y_pred)

print(f"Accuracy Final (Random Forest): {rf_accuracy:.4f}")

Accuracy Final (Random Forest): 0.7318


## *4.2. Regresión Logística*

In [17]:
log_reg_model = LogisticRegression(solver='liblinear', C=10, random_state=42)

log_reg_model.fit(X_train_vec, y_train) 

log_reg_y_pred = log_reg_model.predict(X_test_vec)
log_reg_accuracy = accuracy_score(y_test, log_reg_y_pred)

print(f"Accuracy Final (Regresión Logística): {log_reg_accuracy:.4f}")

Accuracy Final (Regresión Logística): 0.8268


# **5. Comparativa de Modelos**

In [19]:
df_final_data = {
    'Modelo': ['MultinomialNB (Optimizado)', 'Regresión Logística', 'Random Forest'],
    'Accuracy': [accuracy_score(y_test, best_mnb_model.predict(X_test_vec)), 
                 log_reg_accuracy, 
                 0.7318] # Valor previamente calculado
}

df_comparacion = pd.DataFrame(df_final_data).sort_values(by='Accuracy', ascending=False).round(4)

print("\n Rendimiento de Clasificadores")
print(df_comparacion.to_string(index=False))


 Rendimiento de Clasificadores
                    Modelo  Accuracy
MultinomialNB (Optimizado)    0.8268
       Regresión Logística    0.8268
             Random Forest    0.7318


# **6. Conclusiones Finales: Mejor Modelo**

El mejor modelo para esta tarea es el **Multinomial Naive Bayes (MNB) Optimizado**, que alcanzó un `Accuracy` del 0.8268. Este resultado es la validación perfecta de la teoría, que postula que el MNB es el clasificador ideal para este tipo de datos de recuento de palabras. Además del resultado, el MNB suele ser la decisión acertada ya que tiene una mayor simplicidad y velocidad de entrenamiento en PROD.

Alternativa Válidas: 
- Se propuso la **Regresión Logística** como la alternativa más fuerte para superar al Naive Bayes.
  - *Resultado*: La Regresión Logística igualó el rendimiento del MNB (0.8268).
  - *Argumento*: Los modelos lineales, son extremadamente efectivos en clasificación de texto debido a su capacidad para aprender las ponderaciones de forma aislada en espacios de alta dimensión